Create Test Pieces
===============
Creates pdf pages of pieces for human evaluation.

In [134]:
import os
from music21 import *

In [135]:
real_data = "" #file containing original pieces

In [136]:
lstm_files = "" #model folder being evaluated

In [138]:
test_data = []

In [139]:
streams = []
file_names = []
for file in os.listdir(real_data):
    if "mxl" in file:
        streams.append(converter.parse(real_data+file))
        file_names.append(file)
print(len(file_names))

47


In [140]:
def remove_unwanteds(parts, file_names):
    cleaned_parts = []
    new_file_names = []
    for i, a in enumerate(parts):
        a = instrument.partitionByInstrument(a)[0]
        if a.timeSignature.ratioString == "6/8":
            a.remove(list(a.getElementsByClass("TextExpression")))
            a.remove(list(a.getElementsByClass("Dynamic")))
            a.remove(list(a.getElementsByClass("Dynamics")))
            a.remove([x for x in a.getElementsByClass("Spanner") if not isinstance(x, spanner.Slur)])
            for x in a.getElementsByClass("Note"):
                x.articulations=[]
                x.expressions=[]
            cleaned_parts.append(a)
            new_file_names.append(file_names[i])
    return cleaned_parts, new_file_names

In [141]:
cleaned_streams, file_names = remove_unwanteds(streams, file_names)

In [142]:
real_piece_folder = lstm_files + "real_pieces"
try:
    os.mkdir(real_piece_folder)
except:
    pass
for p,f in zip(cleaned_streams, file_names):
    p.write('xml', fp='{}/{}.xml'.format(real_piece_folder, f))

In [143]:
from fpdf import FPDF
from PIL import Image

import img2pdf

In [144]:
from math import ceil
def write_scores_to_pdf(score_images, no_alpha_image_files, concated_file_names, output_file):
    for si, naif in zip(score_images, no_alpha_image_files):
        image = Image.open(si)
        new_img = Image.new("RGB", (image.size[0],image.size[1]), (255, 255, 255))
        cmp_img = Image.composite(image, new_img, image).quantize(colors=256, method=2)
        cmp_img.save(naif)
    #images_per_page = ceil(len(no_alpha_image_files)/len(concated_file_names))
    images_per_page = 3
    for idx, fn in enumerate(concated_file_names):
        cc_file = get_concat_v(no_alpha_image_files[idx*images_per_page:(idx+1)*images_per_page])
        cc_file.save(fn)
    letter = (img2pdf.in_to_pt(8.27), img2pdf.in_to_pt(11.69))
    layout = img2pdf.get_layout_fun(letter)
    with open(output_file, "wb") as f:
        f.write(img2pdf.convert(concated_file_names, layout_fun=layout, margin=100))
def get_concat_v(images):
    print(images)
    opened_images = [Image.open(i) for i in images]
    top_margin = 200
    side_margin = 50
    spacing = 300
    dst = Image.new('RGB', (opened_images[0].width + 2*side_margin, sum([oi.height for oi in opened_images])+2*top_margin+3*spacing), color=(255,255,255))
    last_img_height = top_margin
    for i in opened_images:
        dst.paste(i, (side_margin, last_img_height))
        last_img_height=i.height + last_img_height + spacing     
    return dst

In [145]:
import random
def get_test_print_outs(output_folder):
    folders = [x[0] for x in os.walk(output_folder)]
    try:
      folders.remove(output_folder)
    except ValueError:
      pass
    print(folders)
    test_files = []
    files = []
    for f in folders:
        f_files = []
        for i, file in enumerate(random.sample(list(os.listdir(f)), 18)):
            if ".xml" not in file:
                continue
            conv = converter.subConverters.ConverterMusicXML()
            # conv.write(parts_paths[0], fmt='musicxml.png', fp='file', subformats=['png'])
            piece = converter.parse(f+"/"+file)
            files.append(file)
            instrument_parts = instrument.partitionByInstrument(piece)
            violin_part = instrument_parts[0]
            image_file = violin_part.write("musicxml.png")
            f_files.append(image_file)
        test_files.append(f_files)
    return test_files, files

def write_to_score(test_files):
    #split_into_pages = [[test_files[0][i], test_files[1][i], test_files[2][i], test_files[3][i]] for i in range(10)]#split_into_pages = [[test_files[0][i], test_files[1][i], test_files[2][i], test_files[3][i]] for i in range(10)]
    split_into_pages = [[test_files[0][i], test_files[1][i], test_files[2][i]] for i in range(18)]
    flat_pages = []
    for s in split_into_pages:
        flat_pages += s
    print(flat_pages)
    write_scores_to_pdf(flat_pages, ["test_score_images/{}".format(f.split("/")[-1]) for f in flat_pages], ["concat_image_{}.png".format(i) for i in range(len(split_into_pages))], "test_files.pdf")
    return split_into_pages
        
    

In [146]:
test_files, files = get_test_print_outs(lstm_files)

['slurs_notes_gs_1/hit_features-sn_augmented_T-lstm_units-512_dropout-0.3_lstm_layers-3_single_loss_function-categorical_crossentropy_multi_loss_function-binary_crossentropy_optimiser-adam_epochs-60_last_layer_inputs-F_Sun-05-Apr-19-23-30/test_results/loss/generated_scores/from_ends', 'slurs_notes_gs_1/hit_features-sn_augmented_T-lstm_units-512_dropout-0.3_lstm_layers-3_single_loss_function-categorical_crossentropy_multi_loss_function-binary_crossentropy_optimiser-adam_epochs-60_last_layer_inputs-F_Sun-05-Apr-19-23-30/test_results/loss/generated_scores/real_pieces', 'slurs_notes_gs_1/hit_features-sn_augmented_T-lstm_units-512_dropout-0.3_lstm_layers-3_single_loss_function-categorical_crossentropy_multi_loss_function-binary_crossentropy_optimiser-adam_epochs-60_last_layer_inputs-F_Sun-05-Apr-19-23-30/test_results/loss/generated_scores/from_artificial_seeds']


In [147]:
test_files_copy = test_files
test_files_copy[0] = test_files_copy[0][:18]
write_to_score(test_files_copy)

18
['/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmp5e_4qqsx-1.png', '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpimp42gmm-1.png', '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpg8zx2186-1.png', '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpkjc182el-1.png', '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpdwhuv0oz-1.png', '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpuxoswjmw-1.png', '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpddixzpgp-1.png', '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpnkxxihh_-1.png', '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpog2rlxzd-1.png', '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpuzgfenhp-1.png', '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpjztoyias-1.png', '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpgd0ue3jn-1.png', '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tm

[['/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmp5e_4qqsx-1.png',
  '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpimp42gmm-1.png',
  '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpg8zx2186-1.png'],
 ['/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpkjc182el-1.png',
  '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpdwhuv0oz-1.png',
  '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpuxoswjmw-1.png'],
 ['/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpddixzpgp-1.png',
  '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpnkxxihh_-1.png',
  '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpog2rlxzd-1.png'],
 ['/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpuzgfenhp-1.png',
  '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpjztoyias-1.png',
  '/var/folders/tt/j5m0x77x21j4d074z9vtrmrc0000gn/T/music21/tmpgd0ue3jn-1.png'],
 ['/var/folders/tt/j5m0x77x21j4d074z